In [2]:
import os
from kb.AN_KnowledgeBase import KnowledgeBaseManager
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

True

In [3]:
cosmos_account_name_full = "https://fyp-chatbot-storage.mongo.cosmos.azure.com:443"
cosmos_account_name = "fyp-chatbot-storage"
cosmos_account_key = "sg6JwCvMNJumXadhLiKXoTrlF0piKBIYxTD71cn5sfbgWBdUJiRNd0dIndcZy2wVVHvkHzcd1tgXACDbjMscBA=="
cosmos_database_id = "qnaDatabase"

cosmosdb_connection_string = "mongodb://fyp-chatbot-storage:sg6JwCvMNJumXadhLiKXoTrlF0piKBIYxTD71cn5sfbgWBdUJiRNd0dIndcZy2wVVHvkHzcd1tgXACDbjMscBA==@fyp-chatbot-storage.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@fyp-chatbot-storage@"
connection_string = f"AccountEndpoint=https://{cosmos_account_name}.documents.azure.com;AccountKey={cosmos_account_key};Database={cosmos_database_id};ApiKind=MongoDb"

In [26]:
from azure.search.documents.indexes import SearchIndexerClient

# Create a data source 
indexer_client = SearchIndexerClient(
                os.environ.get("AZURE_AI_SEARCH_ENDPOINT"),
                AzureKeyCredential(os.environ.get("AZURE_AI_SEARCH_API_KEY")),
            )

In [5]:
index_name = "fyp-test"

In [6]:
# create data source
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

data_source_name = "asknarelle-questions-and-answers-mongodb"
container = SearchIndexerDataContainer(name="questions")
data_source_connection = SearchIndexerDataSourceConnection(
    name=data_source_name,
    type="cosmosdb",
    connection_string=connection_string,
    container=container,
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'asknarelle-questions-and-answers-mongodb' created or updated


In [21]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    EntityRecognitionSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset  
skillset_name = "qna-to-rag-ss-2"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=2000,  
    page_overlap_length=500,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/answer"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    resource_url=os.environ.get("AZURE_OPENAI_ENDPOINT"),  
    deployment_name="text-embedding-3-large",  
    model_name="text-embedding-3-large",
    dimensions=1536,
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")  
    ],  
)
  
index_projections = SearchIndexerIndexProjection(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,  
            parent_key_field_name="parentId",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="content", source="/document/pages/*"),  
                InputFieldMappingEntry(name="content_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),  
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
) 

# cognitive_services_account = CognitiveServicesAccountKey(key=AZURE_AI_MULTISERVICE_KEY)

skills = [split_skill, embedding_skill]

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generate embeddings",  
    skills=skills,  
    index_projection=index_projections,
    # cognitive_services_account=cognitive_services_account
)
  

indexer_client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")

qna-to-rag-ss-2 created


In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    FieldMapping
)

# Create an indexer  
indexer_name = "qna-storage-indexer-2"

indexer_parameters = None

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index list of questions and answers", 
    skillset_name=skillset.name,
    target_index_name=index_name,  
    data_source_name=data_source.name, 
    field_mappings=[
        FieldMapping(source_field_name="id", target_field_name="id"),
        FieldMapping(source_field_name="question", target_field_name="title"),
        FieldMapping(source_field_name="answer",)
    ],
    parameters=indexer_parameters
)  


# Create and run the indexer   
indexer_result = indexer_client.create_or_update_indexer(indexer)  

print(f'indexer_result: {indexer_result}')

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')

indexer_result: {'additional_properties': {'@odata.context': 'https://fyp-chatbot-bernkjy.search.windows.net/$metadata#indexers/$entity'}, 'name': 'qna-storage-indexer-2', 'description': 'Indexer to index list of questions and answers', 'data_source_name': 'asknarelle-questions-and-answers-mongodb', 'skillset_name': 'qna-to-rag-ss-2', 'target_index_name': 'fyp-test', 'schedule': None, 'parameters': None, 'field_mappings': [<azure.search.documents.indexes._generated.models._models_py3.FieldMapping object at 0x12c4ba170>, <azure.search.documents.indexes._generated.models._models_py3.FieldMapping object at 0x12c5f9eb0>], 'output_field_mappings': [], 'is_disabled': False, 'e_tag': '"0x8DD3087600E160F"', 'encryption_key': None}
 qna-storage-indexer-2 is created and running. Give the indexer a few minutes before running a query.


In [23]:
indexer_client.run_indexer(indexer.name)

From ChatGPT

In [24]:
# Define the skillset
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    SearchIndexerSkillset,
)

# Create a skillset
skillset_name = "qna-to-rag-ss"

split_skill = SplitSkill(
    description="Split skill to chunk documents",
    text_split_mode="pages",
    context="/document",
    maximum_page_length=2000,
    page_overlap_length=500,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/question"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="textItems", target_name="pages")
    ],
)

embedding_skill = AzureOpenAIEmbeddingSkill(
    description="Skill to generate embeddings via Azure OpenAI",
    resource_url=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    deployment_name="text-embedding-3-large",
    model_name="text-embedding-3-large",
    dimensions=1536,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*"),
    ],
    outputs=[
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")
    ],
)

skills = [split_skill, embedding_skill]

skillset = SearchIndexerSkillset(
    name=skillset_name,
    description="Skillset to chunk documents and generate embeddings",
    skills=skills,
    # cognitive_services_account=CognitiveServicesAccountKey(key=AZURE_AI_MULTISERVICE_KEY)
)

indexer_client.create_or_update_skillset(skillset)
print(f"{skillset.name} created")

qna-to-rag-ss created


In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataSourceType,
    SearchIndexerSkillset,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode
)


# Define data source
data_source_name = "asknarelle-questions-and-answers-mongodb"
container = SearchIndexerDataContainer(name="questions")
# data_source_connection = SearchIndexerDataSourceConnection(
#     name=data_source_name,
#     type="cosmosdb",
#     connection_string=connection_string,
#     container=container,
# )


data_source_connection = SearchIndexerDataSourceConnection(
    name=data_source_name,
    type="cosmosdb",
    connection_string=os.environ.get("COSMOSDB_CONNECTION_STRING"),
    container=container
)

data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

index_projections = SearchIndexerIndexProjection(
    selectors=[
        SearchIndexerIndexProjectionSelector(
            target_index_name=index_name,
            parent_key_field_name="parentId",
            source_context="/document/pages/*",
            mappings=[
                InputFieldMappingEntry(name="content", source="/document/pages/*"),
                InputFieldMappingEntry(name="content_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),
            ],
        ),
    ],
    parameters=SearchIndexerIndexProjectionsParameters(
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS
    ),
)

indexer = SearchIndexer(
    name="qna-indexer",
    data_source_name=data_source_connection.name,
    target_index_name="fyp-test",
    skillset_name=skillset_name,
    field_mappings=index_projections
)

indexer_client.create_or_update_indexer(indexer)
print(f"{indexer.name} created")

index_projections is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.SearchIndexer'> and will be ignored


qna-indexer created
